In [85]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [170]:
data = pd.read_csv('data/matched_routs.csv')

In [171]:
columns_to_zero = ['gps_delta', 'gps_distance', 'gps_speed', 'tm_delta', 'tm_distance', 'tm_speed', 'tm_accel', 'diff_time', 
                   'diff_distance', 'diff_speed', 'ac_delta', 'ac_distance', 'ac_speed', 'delta']
for route_id in tqdm(data['route_id'].unique()):
    route_data = data[data['route_id'] == route_id]
    # route_data['gps_time'] = pd.to_datetime(route_data['gps_time'])
    # route_data.sort_values('gps_time', inplace=True)
    # route_data.reset_index(drop=True, inplace=True)
    route_data['delta'] = pd.to_datetime(route_data['gps_time']).diff().dt.total_seconds().fillna(0)
    route_data['num'] = np.where((route_data['delta'] > 5 * 60) | (route_data.index == route_data.index[0]), route_data.index, np.nan)
    data.loc[route_data[~route_data['num'].isna()].index, columns_to_zero] = 0
    route_data['num'] = route_data['num'].ffill()
    route_data['new_route_id'] = str(route_id) + '_' + route_data['num'].astype(str)
    data.loc[route_data.index, 'new_route_id'] = route_data['new_route_id']

100%|██████████████████████████████████████████████████████████████████████████████| 1068/1068 [06:54<00:00,  2.58it/s]


In [175]:
data.drop(['delta'], axis=1, inplace=True)

In [178]:
data.to_csv('data/routes_new_id.csv', index=False)